In [1]:
import pandas as pd
import numpy as np
import import_ipynb
import sys
import math 
import seaborn as sea
import datetime as dt
import Fuzzy_Prediction_Beta as fz
from datetime import timedelta
from operator import itemgetter
from functools import partial
url = "in.csv"

dataset = pd.read_csv(url, error_bad_lines=False)

selected = dataset[['cod_location','date_id','total_deaths']]

selected = selected.groupby('cod_location').tail(5)

selected.head()
#Input na rede neural
selected.to_csv('input_d.csv')

def aver(input):
    medias = input.groupby('cod_location')['total_deaths'].apply(lambda x: x.tail(5).mean())
    return medias

def diff(input):
    difference = input.groupby('cod_location')['total_deaths'].apply(lambda x: x.tail(1).values[0] - x.head(1).values[0])
    return difference

def first(input):
    first = selected.groupby('cod_location')['total_deaths'].apply(lambda x: x.head(1).values[0])
    return first

def last(input):
    last = selected.groupby('cod_location')['total_deaths'].apply(lambda x: x.tail(1).values[0])
    return last

def param(last,diffe):
    param = last + diffe
    return param

media = aver(selected)
diffe = diff(selected)
first = first(selected)
last = last(selected)
param =  last + diffe
maximus = max(selected['total_deaths'])

fuzzy_data = pd.DataFrame()

importing Jupyter notebook from Fuzzy_Prediction_Beta.ipynb


In [2]:
## Códigos: 0-Queda Brusca 1-Queda 2-Platô 3-Subida 4-Subida Brusca
for i in range(213):
    Regras = fz.verifRegras(param[i], media[i])
    prev = fz.defuzzyfication(np.argmax(Regras))
    per  = fz.getPercentage(Regras)
    centroid = fz.getCentroid(media[i],Regras,param[i])
    fuzzy_data = fuzzy_data.append({'cd_local':i,'id_death_prev':prev,'percent':per,'previsao_numerica':centroid},ignore_index=True)
    fuzzy_data['cd_local'] = fuzzy_data['cd_local'].astype(int)
    fuzzy_data['id_death_prev'] = fuzzy_data['id_death_prev'].astype(int)

In [3]:
fuzzy_data.to_csv('predict_deaths.csv', sep=';', decimal=',')